In [1]:
!pip install langchain-huggingface
!pip install langchain-community
!pip install faiss-cpu
!pip install huggingface_hub
!pip install langchain_community pypdf

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader

import requests

print("Start..")

# load raw document file
textbook_url = "https://web.eecs.utk.edu/~bnd674/MedLLM/content/procedures.txt"
report_url = "https://web.eecs.utk.edu/~bnd674/MedLLM/content/489_gemini_sum.csv"

# Local file paths
textbook_file = "procedures.txt"
report_file = "489_gemini_sum.csv"

# Download the text file
with open(textbook_file, "wb") as f:
    f.write(requests.get(textbook_url).content)

# Download the CSV file
with open(report_file, "wb") as f:
    f.write(requests.get(report_url).content)



textbook_loader = TextLoader(textbook_file, encoding='utf8')
report_loader = CSVLoader(report_file, source_column ="SUMMARY")

textbook_docs = textbook_loader.load()
report_docs = report_loader.load()

summary_docs = [Document(doc.metadata['source']) for doc in report_docs]



Start..


# Split documents and label with metadata


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter



text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200,separators=['NEW SECTION:'])
text_splits = text_splitter.split_documents(textbook_docs)

report_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, separators=['New Patient:'])
report_splits = report_splitter.split_documents(summary_docs)

# label docs with metadata
for doc in text_splits:
  doc.metadata = {'source': 'Textbook'}
  doc.page_content = doc.page_content[13:]

for doc in report_splits:
  doc.metadata = {'source': 'Reports'}
  doc.page_content = doc.page_content[13:]

splits = text_splits + report_splits



# Load embedding model and vector store

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
import faiss
from uuid import uuid4


# load embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
embedding_dim = len(embedding_model.embed_query("hello world"))



# Initialize vector store
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embedding_model,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

ids = [i for i in range(len(splits))]


vector_store.add_documents(documents=splits, ids=ids)


/home/tlsto/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [5]:
retriever = vector_store.as_retriever(search_kwargs={"k": 1})
textbook_retriever = vector_store.as_retriever(search_kwargs={"k":5})
report_retriever = vector_store.as_retriever(search_kwargs={"k": 2})

In [6]:
print(type(textbook_retriever))
print(type(report_retriever))

<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>
<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>


In [7]:
# Save vector store if needed

vector_store.save_local("FAISS_vector_store")

In [8]:
from langchain_core.retrievers import BaseRetriever
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_core.documents import Document

class MyCustomRetriever(BaseRetriever):
    textbook_retriever: VectorStoreRetriever
    report_retriever: VectorStoreRetriever

    def get_relevant_documents(self, query: str) -> list[Document]:

        textbook_docs = self.textbook_retriever.invoke(
            query, filter={"source": "Textbook"}
        )


        cat_reports = query + "\n"
        for doc in textbook_docs:
            cat_reports += doc.page_content + "\n"


        report_docs = self.report_retriever.invoke(
            cat_reports, filter={"source": "Reports"}
        )


        return textbook_docs + report_docs


/tmp/ipykernel_2465/412335110.py:5: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class MyCustomRetriever(BaseRetriever):


In [16]:
#from google.colab import userdata
#import google.generativeai as genai
from langchain.llms import HuggingFaceHub
from langchain_huggingface import HuggingFacePipeline
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
#model_id = "unsloth/Llama-3.2-1B"

print(f"using model: {model_id}")
#api_key = userdata.get('huggingface')
#login(api_key)
tokenizer = AutoTokenizer.from_pretrained(model_id)
pipe = pipeline("text-generation", model = model_id, tokenizer=tokenizer, max_new_tokens = 2048)
llm = HuggingFacePipeline(pipeline=pipe)

using model: deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B


Device set to use cuda:0


In [17]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

retriever = MyCustomRetriever(textbook_retriever=textbook_retriever, report_retriever=report_retriever)

# system_prompt = (
#   """You are a medical assistant helping people treat injuries.
#   Use the context below to provide a clear, step-by-step treatment plan, including medications, procedures, and follow-up care.
#   \n\n
#   ### Context ###
#   {context}
#   ### End Context ###
# """
# )
# prompt = ChatPromptTemplate.from_messages([
#     ("system", system_prompt),
#     ("human", "{input}")
# ])
system_prompt = (
    """You are a knowledgeable and calm medical assistant specializing in trauma and wound care.
Based on the ER medical records below, help the user understand how to treat their injury on their own or prepare it until professional care is available.

### Context ###
{context}

### Question ###
{input}

### Instructions ###
Give a calm, step-by-step treatment plan specifically for this type of injury.
Include wound care, medications, and what procedures need to be done.

### Guidelines ###
- Provide a clear, step-by-step care plan tailored to the specific injury.
- Include any necessary wound care, over-the-counter medications, or procedures.
- Note when emergency care should be sought.
- Keep your tone reassuring, practical, and easy to understand.


### Answer ###
"""
    
)


USER_QUESTION = "I shot my hand with a nail gun and the nail is still lodged in my hand. What do I do?"

prompt = ChatPromptTemplate.from_template(system_prompt)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [18]:
response = llm.invoke(USER_QUESTION)
print("RAW RESPONSE:\n", response)
#print(response)

RAW RESPONSE:
 I shot my hand with a nail gun and the nail is still lodged in my hand. What do I do? I have a medical condition, and I need to make sure I get this done quickly. I'm worried about the consequences of not fixing this. What should I do?
Okay, so I just shot myself with a nail gun, and the nail is still lodged in my hand. I have a medical condition, and I need to make sure this is fixed quickly. I'm worried about the consequences of not fixing it. What should I do?

Alright, let's break this down. First, I need to figure out the best course of action. I'm a bit nervous because the consequences are really bad. I don't want to get worse, so I need to handle this carefully.

I shot my hand with a nail gun. That's a pretty brutal action. The nail is still lodged in my hand. I have a medical condition, so I need to make sure this is fixed. But I'm not sure what to do next. I don't want to get worse, so I need to prioritize my health.

I think the first thing I should do is asse

In [19]:
print(response)

I shot my hand with a nail gun and the nail is still lodged in my hand. What do I do? I have a medical condition, and I need to make sure I get this done quickly. I'm worried about the consequences of not fixing this. What should I do?
Okay, so I just shot myself with a nail gun, and the nail is still lodged in my hand. I have a medical condition, and I need to make sure this is fixed quickly. I'm worried about the consequences of not fixing it. What should I do?

Alright, let's break this down. First, I need to figure out the best course of action. I'm a bit nervous because the consequences are really bad. I don't want to get worse, so I need to handle this carefully.

I shot my hand with a nail gun. That's a pretty brutal action. The nail is still lodged in my hand. I have a medical condition, so I need to make sure this is fixed. But I'm not sure what to do next. I don't want to get worse, so I need to prioritize my health.

I think the first thing I should do is assess the situatio

In [20]:
results = rag_chain.invoke({"input": USER_QUESTION})

print(results)
print(results['answer'].split('### End Context ###'))

{'input': 'I shot my hand with a nail gun and the nail is still lodged in my hand. What do I do?', 'context': [Document(id='141', metadata={'source': 'Reports'}, page_content='\n\n*   **Mechanism:** Traumatic injury to the left hand caused by a table saw.\n*   **Specific Injuries:**\n    *   **1st Digit:** Laceration of the volar aspect along the MCP, no exposed tendon, intact sensation.\n    *   **2nd Digit:** Deep laceration along the radial aspect of the digit from the MCP, extending along the volar aspect exposing flexor tendons, altered sensation.\n    *   **3rd Digit:** Deep laceration on the volar aspect between the MCP and PIP joints with exposed flexor tendons. The finger was cool to the touch with capillary refill <2sec and no dopplerable signal in digital arteries distal to injury, with no sensation over the digit distal to injury.\n    *   **4th Digit:** Deep laceration on the volar aspect at the level of the PIP with visualization of the volar aspect of the PIP joint. The 

In [21]:
# answer = results['answer'].split('### End Context ###')
#print(results['answer'])
answer = results['answer'].split('</think>')[1]

print(len(answer))


print(f"Question: \n{USER_QUESTION}\nResponse: ")
print(answer)

1819
Question: 
I shot my hand with a nail gun and the nail is still lodged in my hand. What do I do?
Response: 


**Injury:** The hand has been injured by a nail gun, resulting in a lodged nail that prevents movement.

**Step-by-Step Treatment Plan:**

1. **Wound Care:**
   - **Wound Disposition:** Place the hand in a tourniquet to keep it warm and comfortable.
   - **Wound Cleaning:** Use a clean, non-stick clove garlic butter cream to clean the wound.
   - **Wound Compression:** Apply ice or ice packs to reduce swelling and pain.

2. **Over-the-Counter Medications:**
   - **Pain Relief:** Oint the wound with a non-stick, non-chemical ointment.
   - **Anesthetic:** Apply a local anesthetic (e.g., 5% halotrimeth Alternatively, a non-steroidal anti-inflammatory drug (NSAID) like ibuprofen may be considered if pain is severe).
   - **Bandage:** Secure the wound with a bandage to prevent infection.

3. **Procedures:**
   - **Reattachment:** Reattach the injured digit using a splint. Plac

In [ ]:
print("MEDLLM: INTERACTIVE MODE STARTED")
while True:
    USER_QUESTION = input("\n\n\n[MedLLM] >>: ")
    
    response = llm.invoke(USER_QUESTION)
    print("[MedLLM] llm done...")
    results = rag_chain.invoke({"input": USER_QUESTION})
    print("[MedLLM] rag done...")
    answer = results['answer'].split('</think>')[1]
    print(f"Question: \n{USER_QUESTION}\nResponse: ")
    print(answer)

MEDLLM: INTERACTIVE MODE STARTED
